In [1]:
#!/usr/bin/env python
# coding: utf-8
import warnings
warnings.filterwarnings("ignore")

import sys
import os
import datetime
import pickle
import io
import re
import code

from chainer import no_backprop_mode, cuda, Variable
from chainer.functions import relu
import cupy as cp
import numpy as np
import scipy.stats as stats
import statsmodels.stats.proportion as prop
from exp_settings import settings
import trainers.trainer as trainer
import measurements.robustness as robustness

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # In case the GPU order in the bus is different from the one listed by CUDA


In [2]:
def change_gpu(model, gpu):
    """ Used in case the saved net was in a different GPU from the one used when loading
    Args:
        model: NNAgent object
        gpu: number of the GPU to be transfered to

    """
    for link in range(len(model)):
        if hasattr(model[link], 'W'):
            model[link].W.to_gpu(gpu)
            model[link].ortho_w.to_gpu(gpu)
        if hasattr(model[link], 'b') and model[link].b is not None:
            model[link].b.to_gpu(gpu)

class RenameUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        renamed_module = module
        return super(RenameUnpickler, self).find_class(renamed_module, name)

class RenameUnpickler2(pickle.Unpickler):
    def find_class(self, module, name):
        renamed_module = module
        if module == "BNN_mod_bu":
            renamed_module = "BNN_mod_normal"
        return super(RenameUnpickler2, self).find_class(renamed_module, name)

def renamed_load(file_obj):
    return RenameUnpickler(file_obj).load()

def renamed_loads(pickled_bytes):
    file_obj = io.BytesIO(pickled_bytes)
    return renamed_load(file_obj)

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = renamed_load(fo)
    return dict

def experiment_routine(**kwargs):
    """ Function to be called to carry multiple trainings under the same settings

    Args:
        **kwargs:

    Returns: kwargs dictionary containing the number of successful and failed trainings

    """
    # Creates string to describe whether the input is normalized across all data or just rescaled to specified interval
    if kwargs['normalization']:
        in_str = "norm_in"
    else:
        in_str = "in_int_{}_cent_{}".format(kwargs['in_interval'], kwargs['in_center'])

    arch = kwargs['arch']
    date = datetime.datetime.today().strftime('%Y-%m-%d')

    curr_dir, curr_fold = os.path.split(os.path.dirname(os.path.realpath(__file__)))

    if kwargs['bjorck_config']['iter'] == 0:
        dest_dir = curr_dir + "/trainings/{}/{}/no_ortho/loss_{}/init_{}/{}/{}_tr/{}/".format(
        kwargs['dataset'], arch, kwargs['loss'], kwargs['init'], in_str, kwargs['tr_size'], date)
    else:
        dest_dir = curr_dir + "/trainings/{}/{}/ortho/loss_{}/init_{}/{}/{}_tr/{}/".format(
        kwargs['dataset'], arch, kwargs['loss'], kwargs['init'], in_str, kwargs['tr_size'], date)
    print("Destination folder \n{}".format(dest_dir))

    j = 0
    n_exp = kwargs['n_exp']
    fail = 0
    success = 0
    while success < n_exp:
        # Sets the name of the destination folder for current training with the main training settings
        exp_name = "loss_{}_ep_{}_lr_{}_x_var_{}_d_{}_{}".format(kwargs['loss'], kwargs['n_epoch'], kwargs['lr'], kwargs['x_var'], kwargs['d'], j)
        # Changes the number tag of destination folder until a number that wasn'target used
        while os.path.exists(dest_dir + exp_name):
            j += 1
            exp_name = "loss_{}_ep_{}_lr_{}_x_var_{}_d_{}_{}".format(kwargs['loss'], kwargs['n_epoch'], kwargs['lr'],  kwargs['x_var'], kwargs['d'], j)

        data_save_dir = dest_dir + exp_name
        os.makedirs(data_save_dir)

        kwargs_train = {'exp_count': success + fail, 'success': success, 'data_save_dir': data_save_dir}
        kwargs = {**kwargs, **kwargs_train}
        print("#### Training {}".format(success + fail + 1))
        print("## X_var: {} | d: {} | lr: {} | tr_size: {}".format(kwargs['x_var'], kwargs['d'], kwargs['lr'], kwargs['tr_size']))
        tr_result = trainer.run_training(**kwargs)

        if tr_result == 0:
            fail += 1
        else:
            success += 1

    print("*****************************************************************")
    print("The number of fails = {} and successes = {}".format(fail, success))
    print("*****************************************************************")

    return kwargs

def build_net(**kwargs):
    """Builds a base agent that will be used to load the trained NN
    Make sure that all the architecture settings match
    """
    network = trainer.NNAgent(**kwargs)
    network.prepare_data(**kwargs)
    network.create_model(**kwargs)
    return network


In [3]:
mode = "load"
gpu = 3
cp.cuda.Device(gpu).use()     
loss = "zhen"
d = cp.asarray(4.0, dtype=cp.float32)
x_var = cp.asarray(0.0001, dtype=cp.float32)

input_params = {'loss': loss, 'd': d, 'x_var': x_var, 'gpu': gpu}

kwargs = settings(input_params)

Architecture used: 4CVHL_16c4k2sx16c3k1sx64c4k2sx64c3k1sx_1FCHL_512


In [4]:
_, curr_fold = os.path.split(kwargs['net_save_dir'])
curr_dir = os.getcwd()
measures_save_dir = curr_dir + "/measurements/{}/".format(kwargs['dataset']) + "{}/{}/".format(kwargs['arch'],curr_fold)
print(measures_save_dir)
if not os.path.isdir(measures_save_dir[0:-1]):
    os.makedirs(measures_save_dir[0:-1])

curr_dir, curr_fold = os.path.split(curr_dir)
filelist = robustness.list_nets(kwargs['net_save_dir'], loss = loss, d = d, x_var = x_var, load_mode = mode)
print(filelist)
# loops through all files in the .csv file
file = filelist[0]
net_path = file
print(kwargs['net_save_dir'])
print(measures_save_dir)
print(file)
kwargs_load = {'exp_count': None, 'success': None, 'data_save_dir': None}
kwargs = {**kwargs, **kwargs_load}
# Builds NNAgent object with architecture settings
network = build_net(**kwargs)
# Load trained NN into the NNAgent object
network.model = unpickle(net_path)
# Changes GPU in case the GPU used during training is different from the one used from here
change_gpu(network.model, gpu)
# Prepares data with same settings as ones used for training
network.prepare_data(mode='load', **kwargs)

# Data used for measurements are TEST DATA. Here it changes it to Variable object so that
# differentiation w.r.t. inputs can be done
x_m = Variable(network.te_x)
target = Variable(network.te_y)

# Parses the trained NN settings type of loss, x_var, d, training epochs and training number from the file name
# code.interact(local=locals())
loss = re.search('loss_(.*)_ep', net_path).group(1)
epochs = re.search('_ep_(.*)_x_var', net_path).group(1)
x_var = re.search('x_var_(.*)_d', net_path).group(1)
d = re.search('{}_d_(.*)_'.format(x_var), net_path).group(1)
training_num = float(re.search('_d_{}_(.*)'.format(d), net_path).group(1))
x_var = float(x_var)
d = float(d)

arch = network.model.arch
network.loss = loss
network.epochs = epochs
network.x_var = x_var
network.d = d
network.model.intvl_in = network.intvl_in
network.model.center_in = network.center_in

# Sets the Bjorck orthogonalization settings for each layer
for j in range(len(network.model)):
    if hasattr(network.model[j], 'W'):
        network.model[j].config['dynamic_iter'] = True
        network.model[j].dynamic_iter = True

network.model.ortho_iter_red()

/home/thomas/sra/measurements/cifar10/cnn/4CVHL_16c4k2sx16c3k1sx64c4k2sx64c3k1sx_1FCHL_512/
### Measurements will be carried for zhen loss, d = 4.0 and x_var = 1e-04
['/home/thomas/trained_NNs/cifar10/cnn/4CVHL_16c4k2sx16c3k1sx64c4k2sx64c3k1sx_1FCHL_512/trained_loss_zhen_ep_300_x_var_1e-04_d_4.0_2', '/home/thomas/trained_NNs/cifar10/cnn/4CVHL_16c4k2sx16c3k1sx64c4k2sx64c3k1sx_1FCHL_512/trained_loss_zhen_ep_300_x_var_1e-04_d_4.0_1', '/home/thomas/trained_NNs/cifar10/cnn/4CVHL_16c4k2sx16c3k1sx64c4k2sx64c3k1sx_1FCHL_512/trained_loss_zhen_ep_300_x_var_1e-04_d_4.0_3']
/home/thomas/trained_NNs/cifar10/cnn/4CVHL_16c4k2sx16c3k1sx64c4k2sx64c3k1sx_1FCHL_512
/home/thomas/sra/measurements/cifar10/cnn/4CVHL_16c4k2sx16c3k1sx64c4k2sx64c3k1sx_1FCHL_512/
/home/thomas/trained_NNs/cifar10/cnn/4CVHL_16c4k2sx16c3k1sx64c4k2sx64c3k1sx_1FCHL_512/trained_loss_zhen_ep_300_x_var_1e-04_d_4.0_2
Using zhen loss
#### Preparing dataset
Dataset: cifar10 with 50000 training samples


Dataset file to be loaded:
/home/thomas/datasets/dataset_cifar10/cifar10_preprocessed_data_1.0_0.5.npy
#### Finished data preparation
############# Number of epochs: 300
NN with orthogonal layers
### Generating NN model
# If this is the first time running it can take several minutes. Please wait
20
Mean pairwise dot offdiag:  1.41372665e-08
Mean pairwise dot diag:  1.0
19
Mean pairwise dot offdiag:  2.4897629e-08
Mean pairwise dot diag:  0.9999998
18
Mean pairwise dot offdiag:  5.6274635e-06
Mean pairwise dot diag:  0.9999466
18
Mean pairwise dot offdiag:  5.6274635e-06
Mean pairwise dot diag:  0.9999466
19
Mean pairwise dot offdiag:  2.4897629e-08
Mean pairwise dot diag:  0.9999998
20
12
Mean pairwise dot offdiag:  1.5684344e-08
Mean pairwise dot diag:  1.0
11
Mean pairwise dot offdiag:  4.5702013e-08
Mean pairwise dot diag:  1.0
10
Mean pairwise dot offdiag:  1.3312253e-05
Mean pairwise dot diag:  0.9999611
10
Mean pairwise dot offdiag:  1.3312253e-05
Mean pairwise dot diag:  0.99996

In [5]:
def sampleundernoise(network, x, sd, times, dist = 'Normal', radius = 0.3):
    # Using no_backprop_mode method since it is just required to forward the input through the network, and not to build a computational graph to apply the backprop
    n = 50000
    count_vec = network.model.xp.zeros(10)
    with no_backprop_mode():
        x = network.model.xp.repeat(cp.expand_dims(x.array,0), n, 0)
        for _ in range(times):
            noise = network.model.noise_inj(x, sd, dist, radius)
            h = x + noise
            for link in range(len(network.model) - 1):
                if hasattr(network.model[link], 'W'):
                    h = relu(network.model[link](h))
                else:
                    h = network.model[link](h)
            h = network.model[link + 1](h)
            shift_out = h.array - h.array.min()
            max_out = network.model.xp.max(shift_out, axis = 1).reshape((n,1))        
            aux = (shift_out/max_out).astype(dtype=cp.int8)
            count_vec += network.model.xp.sum(aux, axis = 0)
    return count_vec

def certify(network, x_in, sd, times, alpha):
    # count_mat0 = cuda.to_cpu(sampleundernoise(network, x_in, sd, n0))
    # cA = np.argmax(count_mat0,axis = 1)
    count_mat = cuda.to_cpu(sampleundernoise(network, x_in, sd, times))
    cA = np.argmax(count_mat)
    nA = count_mat[cA]
    normal = stats.norm
    p = prop.proportion_confint(nA, times*50000, alpha = 2*alpha, method = "beta")[0]
    if p > 0.5:
        radius = sd*normal.ppf(p)
    else:
        cA = -1
        radius = -1
    return cA, radius

In [6]:
count = 0
for i in range(100):
    idx = np.random.choice(np.arange(10000))
    # count_mat = sampleundernoise(network, x_m[idx], 0.3, 45000)
    cA, radius = certify(network, x_m[idx], 0.0001, 1, 0.05)
    if target.array[idx] == cA:
        count += 1
# print(target.array[idx])
# print(cA)
# print(radius)

Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished sampling
Finished s

In [11]:
out_samples = cuda.to_cpu(network.model.output_sampling(x_m, 0, 1))
class_out = np.argmax(out_samples, axis = 1)
tgt = cuda.to_cpu(target.array)
corr_in = x_m[class_out==tgt]
corr_idx = np.arange(10000)[class_out==tgt]
cr_arr = []
for i in corr_idx:
    sd = 0.02
    cr_arr_temp = []
    cA, radius = certify(network, x_m[i], sd, 1, 0.05)
    print("sd {} radius {}".format(sd, radius))
    while cA == tgt[i] and sd < 0.4:
        cr_arr_temp.append(radius)
        sd += 0.02
        cA, radius = certify(network, x_m[i], sd, 1, 0.05)
        print("sd {} radius {} class {}".format(sd, radius, cA))
        
    if sd == 0.02:
        sd = 0.002
        cA, radius = certify(network, x_m[i], sd, 1, 0.05)
        print("sd {} radius {}".format(sd, radius))
        while cA == tgt[i] and sd < 0.02:
            cr_arr_temp.append(radius)
            sd += 0.002
            cA, radius = certify(network, x_m[i], sd, 1, 0.05)
            print("sd {} radius {} class {}".format(sd, radius, cA))
    
    cr_arr.append(np.asarray(cr_arr_temp).max())


Finished sampling
sd 0.02 radius 0.07692964933353241
Finished sampling
sd 0.04 radius 0.15385929866706483 class 3
Finished sampling
sd 0.06 radius 0.2307889480005972 class 3
Finished sampling
sd 0.08 radius 0.30771859733412965 class 3
Finished sampling
sd 0.1 radius 0.38464824666766206 class 3
Finished sampling
sd 0.12000000000000001 radius 0.4615778960011945 class 3
Finished sampling
sd 0.14 radius 0.4730204461088414 class 3
Finished sampling
sd 0.16 radius 0.32193190968555796 class 3
Finished sampling
sd 0.18 radius 0.14373960587498166 class 3
Finished sampling
sd 0.19999999999999998 radius 0.0419066245842109 class 6
Finished sampling
sd 0.02 radius 0.07692964933353241
Finished sampling
sd 0.04 radius 0.15385929866706483 class 8
Finished sampling
sd 0.06 radius 0.2307889480005972 class 8
Finished sampling
sd 0.08 radius 0.30771859733412965 class 8
Finished sampling
sd 0.1 radius 0.38464824666766206 class 8
Finished sampling
sd 0.12000000000000001 radius 0.42760458567569987 class 8
Fi

Finished sampling
sd 0.1 radius 0.1692299271875089 class 8
Finished sampling
sd 0.12000000000000001 radius 0.09603450569372575 class 8
Finished sampling
sd 0.14 radius 0.015764413168784353 class 8
Finished sampling
sd 0.16 radius 0.061341004488592754 class 1
Finished sampling
sd 0.02 radius 0.07692964933353241
Finished sampling
sd 0.04 radius 0.15385929866706483 class 0
Finished sampling
sd 0.06 radius 0.2307889480005972 class 0
Finished sampling
sd 0.08 radius 0.30771859733412965 class 0
Finished sampling
sd 0.1 radius 0.38464824666766206 class 0
Finished sampling
sd 0.12000000000000001 radius 0.26678525279792337 class 0
Finished sampling
sd 0.14 radius 0.058775753889716856 class 0
Finished sampling
sd 0.16 radius -1 class -1
Finished sampling
sd 0.02 radius 0.07692964933353241
Finished sampling
sd 0.04 radius 0.15385929866706483 class 6
Finished sampling
sd 0.06 radius 0.2307889480005972 class 6
Finished sampling
sd 0.08 radius 0.30771859733412965 class 6
Finished sampling
sd 0.1 rad

In [44]:
idx = np.random.choice(np.arange(10000))
x = x_m[idx]
sd = 0.2
n = 50000
dist = 'Normal'
radius = 0.3

with no_backprop_mode():
    count_mat = network.model.xp.zeros((n,10))
    x = network.model.xp.repeat(cp.expand_dims(x.array,0), n, 0)
    noise = network.model.noise_inj(x, sd, dist, radius)
    h = x + noise
    for link in range(len(network.model) - 1):
        if hasattr(network.model[link], 'W'):
            h = relu(network.model[link](h))
        else:
            h = network.model[link](h)
    h = network.model[link + 1](h)
    shift_out = h.array - h.array.min()
    max_out = network.model.xp.max(shift_out, axis = 1).reshape((n,1))        
    aux = (shift_out/max_out).astype(dtype=cp.int8)
    count_mat = network.model.xp.sum(aux, axis = 0)

print(count_mat)
print(count_mat.argmax())
print(target.array[idx])

[    4     0    28     1   150     0 49815     2     0     0]
6
8


In [68]:
count_mat

array([    0,     0,     0,     0, 12763,   264, 30998,   975,     0,
           0])

In [45]:
count_mat

array([    0,     0,     3,     0,     0,     0, 44997,     0,     0,
           0])

In [ ]:
count = 0
for i in range(100):
    idx = np.random.choice(np.arange(10000))
    # count_mat = sampleundernoise(network, x_m[idx], 0.3, 45000)
    cA, radius = certify(network, x_m[idx], 0.0001, 1, 0.05)
    if target.array[idx] == cA:
        count += 1
# print(target.array[idx])
# print(cA)
# print(radius)

CUDARuntimeError: cudaErrorIllegalAddress: an illegal memory access was encountered

Exception ignored in: 'cupy.cuda.memory.Memory.__dealloc__'
Traceback (most recent call last):
  File "cupy/cuda/runtime.pyx", line 349, in cupy.cuda.runtime.free
  File "cupy/cuda/runtime.pyx", line 197, in cupy.cuda.runtime.check_status
cupy.cuda.runtime.CUDARuntimeError: cudaErrorIllegalAddress: an illegal memory access was encountered


CUDARuntimeError: cudaErrorIllegalAddress: an illegal memory access was encountered

Exception ignored in: 'cupy.cuda.memory.Memory.__dealloc__'
Traceback (most recent call last):
  File "cupy/cuda/runtime.pyx", line 349, in cupy.cuda.runtime.free
  File "cupy/cuda/runtime.pyx", line 197, in cupy.cuda.runtime.check_status
cupy.cuda.runtime.CUDARuntimeError: cudaErrorIllegalAddress: an illegal memory access was encountered


CUDARuntimeError: cudaErrorIllegalAddress: an illegal memory access was encountered

...

CUDARuntimeError: cudaErrorIllegalAddress: an illegal memory access was encountered

Exception ignored in: 'cupy.cuda.memory.Memory.__dealloc__'
Traceback (most recent call last):
  File "cupy/cuda/runtime.pyx", line 349, in cupy.cuda.runtime.free
  File "cupy/cuda/runtime.pyx", line 197, in cupy.cuda.runtime.check_status
cupy.cuda.runtime.CUDARuntimeError: cudaErrorIllegalAddress: an illegal memory access was encountered


CUDARuntimeError: cudaErrorIllegalAddress: an illegal memory access was encountered

Exception ignored in: 'cupy.cuda.memory.Memory.__dealloc__'
Traceback (most recent call last):
  File "cupy/cuda/runtime.pyx", line 349, in cupy.cuda.runtime.free
  File "cupy/cuda/runtime.pyx", line 197, in cupy.cuda.runtime.check_status
cupy.cuda.runtime.CUDARuntimeError: cudaErrorIllegalAddress: an illegal memory access was encountered


CUDARuntimeError: cudaErrorIllegalAddress: an illegal memory access was encountered

In [6]:
import numpy as np
from chainer import cuda
import pandas
import pingouin
import scipy.stats as stats
import matplotlib.pyplot as plt

In [7]:
import statsmodels

In [166]:
std = 0.01
samples = 50000
out_samples = cuda.to_cpu(network.model.output_sampling(x_m, 0, 1))
class_out = np.argmax(out_samples, axis = 1)
tgt = cuda.to_cpu(target.array)
corr_in = x_m[class_out==tgt]
in_samples = np.random.randint(0,corr_in.shape[0],100)
histogram = np.zeros(100)
t_arr = []
r_arr = []
for i in in_samples:
	out_samples = network.model.output_sampling(corr_in[i], std, samples)
	avg_out = out_samples.mean(axis = 0)
	df = pandas.DataFrame(data=cuda.to_cpu(out_samples), index=np.arange(samples), columns=np.arange(10))
	table = pingouin.pairwise_corr(df)
	histogram += np.histogram(table['r'], bins = 100, range=(-1,1))[0]
	r_arr.append(table['r'])
	t_arr.append(table['r'] * np.sqrt((samples-2)/(1 - table['r']**2)))
	# cov = 
 	# normal_samples = np.random.multivariate_normal(mean, cov, size=None, check_valid='warn', tol=1e-8)

In [13]:
_, mean_s, var_s, mean_h, var_h = network.model.moment_propagation(len(network.model)-1,corr_in, std**2)

In [59]:
# var_s.shape
corr_in_idx = np.arange(10000)[class_out==tgt]
print(corr_in_idx.shape)
# type(var_s[in_samples[0]])
# out_cov = 

(6609,)


In [55]:
# off_diag = np.logical_not(np.eye(10, dtype = 'bool'))
# corr_out_arr = corr_out[off_diag]
cov_out = network.model[len(network.model)-1].W.array @ cp.diag(var_s[in_samples[0]].array) @ network.model[len(network.model)-1].W.array.T
corr_out = (cp.diag(1/cp.sqrt(cp.diag(cov_out)))) @ cov_out @ (cp.diag(1/cp.sqrt(cp.diag(cov_out))))
corr_out_arr = corr_out[np.tril(corr_out, -1) != 0]

In [9]:
r = np.asarray(r_arr)
t = np.asarray(t_arr)

In [197]:
# out_samples = cuda.to_cpu(out_samples)
import statsmodels

In [198]:
# fig,ax = plt.subplots(1)
idx_aux = np.random.randint(0,100,1)[0]
idx = corr_in_idx[in_samples[idx_aux]]
cov_out = network.model[len(network.model)-1].W.array @ cp.diag(var_s[in_samples[idx_aux]].array) @ network.model[len(network.model)-1].W.array.T
corr_out = (cp.diag(1/cp.sqrt(cp.diag(cov_out)))) @ cov_out @ (cp.diag(1/cp.sqrt(cp.diag(cov_out))))
# statsmodels.stats.moment_helpers.cov2corr(cuda.to_cpu(corr_out))
corr_out_arr = corr_out[np.tril(corr_out, -1) != 0]
print(r[idx_aux])
# print(cuda.to_cpu(corr_out_arr).shape)
fig = plt.figure(dpi=50,constrained_layout=True)
ax = fig.add_subplot(111)
# plt.xlim([-0.012, -0.008])
# plt.ylim([-0.0135, -0.011])
x_axis = np.arange(-1, 1, 0.02)
# line1 = ax.plot(x_axis, histogram)
line1 = ax.hist(r[idx_aux].flatten(), bins=30, range=(-1,1))
line2 = ax.hist(cuda.to_cpu(corr_out_arr), bins=30, range=(-1,1))
# plt.plot(out_samples[:,0],out_samples[:,6])
# fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
# ax.axis('off')
plt.show()

AttributeError: module 'statsmodels' has no attribute 'stats'